# Project: Wrangling and Analyze Data

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import io
import tweepy
import json
from timeit import default_timer as timer

%matplotlib inline

## Data Gathering

### Read the downloaded `twitter_archive_enhanced.csv` file from disk.

In [87]:
df_archive = pd.read_csv('twitter_archive_enhanced.csv')
df_archive.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


### Download `image_predications.tsv` from udacity server

In [5]:
# Using the request library to download 'image_predications.tsv' file online.
image_prediction_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(image_prediction_url)
response.status_code

200

In [6]:
# Get text in file using response.text
response_text = response.text

# Read text data into pandas DataFrame
df_ipred = pd.read_csv(io.StringIO(response_text), sep='\t') # df_ipred -> dataframe_imageprediction
df_ipred.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [7]:
# Save text file locally to 'image_predictions.tsv'
with open('image_predictions.tsv', 'w') as f:
    f.write(response_text)
    
# Making sure file is saved
with open('image_predictions.tsv') as f:
    print(f.readline())

tweet_id	jpg_url	img_num	p1	p1_conf	p1_dog	p2	p2_conf	p2_dog	p3	p3_conf	p3_dog



### Using `tweepy` library to query additional data via Twitter API and save json to `tweet_json.txt`

In [8]:
# Authenticating tweepy api with consumer and access keys and secrets gotten from my twitter developer dashboard.
# Not keys and secrets not included here for security reasons.

consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [14]:
# Downloading and saving each tweet_json in a 'tweet_json.txt' file
#
# NOTE: authenticated 'api' object has been created initially

errors = [] # A list that holds all encountered errors

timer_start = timer()
with open('tweet_json.txt', 'w') as f:
    # Looping through the tweet_ids in the df_archive dataframe and download tweet_json
    for index, tweet_id in enumerate(df_archive.tweet_id):
        try:
            # api.get_status(_id) returns a 'tweepy.models.Status' that I can access using the dot('.') operator
            # while adding a '._json' returns just the json part as a python dictionary
            tweet_json = api.get_status(tweet_id)._json

        except Exception as e:
            # Append error
            errors.append('{}: {}, tweeet_id: {}'.format(type(e).__name__, e.args, tweet_id))

        else:
            # Writing data to file
            json.dump(tweet_json, f)
            f.write('\n')

        finally:
            print('index: {}, tweet_id: {}'.format(index, tweet_id))

timer_end = timer()
print(timer_end - timer_start)

index: 0, tweet_id: 892420643555336193
index: 1, tweet_id: 892177421306343426
index: 2, tweet_id: 891815181378084864
index: 3, tweet_id: 891689557279858688
index: 4, tweet_id: 891327558926688256
index: 5, tweet_id: 891087950875897856
index: 6, tweet_id: 890971913173991426
index: 7, tweet_id: 890729181411237888
index: 8, tweet_id: 890609185150312448
index: 9, tweet_id: 890240255349198849
index: 10, tweet_id: 890006608113172480
index: 11, tweet_id: 889880896479866881
index: 12, tweet_id: 889665388333682689
index: 13, tweet_id: 889638837579907072
index: 14, tweet_id: 889531135344209921
index: 15, tweet_id: 889278841981685760
index: 16, tweet_id: 888917238123831296
index: 17, tweet_id: 888804989199671297
index: 18, tweet_id: 888554962724278272
index: 19, tweet_id: 888202515573088257
index: 20, tweet_id: 888078434458587136
index: 21, tweet_id: 887705289381826560
index: 22, tweet_id: 887517139158093824
index: 23, tweet_id: 887473957103951883
index: 24, tweet_id: 887343217045368832
index: 25,

index: 203, tweet_id: 853299958564483072
index: 204, tweet_id: 852936405516943360
index: 205, tweet_id: 852912242202992640
index: 206, tweet_id: 852672615818899456
index: 207, tweet_id: 852553447878664193
index: 208, tweet_id: 852311364735569921
index: 209, tweet_id: 852226086759018497
index: 210, tweet_id: 852189679701164033
index: 211, tweet_id: 851953902622658560
index: 212, tweet_id: 851861385021730816
index: 213, tweet_id: 851591660324737024
index: 214, tweet_id: 851464819735769094
index: 215, tweet_id: 851224888060895234
index: 216, tweet_id: 850753642995093505
index: 217, tweet_id: 850380195714523136
index: 218, tweet_id: 850333567704068097
index: 219, tweet_id: 850145622816686080
index: 220, tweet_id: 850019790995546112
index: 221, tweet_id: 849776966551130114
index: 222, tweet_id: 849668094696017920
index: 223, tweet_id: 849412302885593088
index: 224, tweet_id: 849336543269576704
index: 225, tweet_id: 849051919805034497
index: 226, tweet_id: 848690551926992896
index: 227, twee

index: 404, tweet_id: 824025158776213504
index: 405, tweet_id: 823939628516474880
index: 406, tweet_id: 823719002937630720
index: 407, tweet_id: 823699002998870016
index: 408, tweet_id: 823581115634085888
index: 409, tweet_id: 823333489516937216
index: 410, tweet_id: 823322678127919110
index: 411, tweet_id: 823269594223824897
index: 412, tweet_id: 822975315408461824
index: 413, tweet_id: 822872901745569793
index: 414, tweet_id: 822859134160621569
index: 415, tweet_id: 822647212903690241
index: 416, tweet_id: 822610361945911296
index: 417, tweet_id: 822489057087389700
index: 418, tweet_id: 822462944365645825
index: 419, tweet_id: 822244816520155136
index: 420, tweet_id: 822163064745328640
index: 421, tweet_id: 821886076407029760
index: 422, tweet_id: 821813639212650496
index: 423, tweet_id: 821765923262631936
index: 424, tweet_id: 821522889702862852
index: 425, tweet_id: 821421320206483457
index: 426, tweet_id: 821407182352777218
index: 427, tweet_id: 821153421864615936
index: 428, twee

index: 604, tweet_id: 798585098161549313
index: 605, tweet_id: 798576900688019456
index: 606, tweet_id: 798340744599797760
index: 607, tweet_id: 798209839306514432
index: 608, tweet_id: 797971864723324932
index: 609, tweet_id: 797545162159308800
index: 610, tweet_id: 797236660651966464
index: 611, tweet_id: 797165961484890113
index: 612, tweet_id: 796904159865868288
index: 613, tweet_id: 796865951799083009
index: 614, tweet_id: 796759840936919040
index: 615, tweet_id: 796563435802726400
index: 616, tweet_id: 796484825502875648
index: 617, tweet_id: 796387464403357696
index: 618, tweet_id: 796177847564038144
index: 619, tweet_id: 796149749086875649
index: 620, tweet_id: 796125600683540480
index: 621, tweet_id: 796116448414461957
index: 622, tweet_id: 796080075804475393
index: 623, tweet_id: 796031486298386433
index: 624, tweet_id: 795464331001561088
index: 625, tweet_id: 795400264262053889
index: 626, tweet_id: 795076730285391872
index: 627, tweet_id: 794983741416415232
index: 628, twee

index: 804, tweet_id: 772117678702071809
index: 805, tweet_id: 772114945936949249
index: 806, tweet_id: 772102971039580160
index: 807, tweet_id: 771908950375665664
index: 808, tweet_id: 771770456517009408
index: 809, tweet_id: 771500966810099713
index: 810, tweet_id: 771380798096281600
index: 811, tweet_id: 771171053431250945
index: 812, tweet_id: 771136648247640064
index: 813, tweet_id: 771102124360998913
index: 814, tweet_id: 771014301343748096
index: 815, tweet_id: 771004394259247104
index: 816, tweet_id: 770787852854652928
index: 817, tweet_id: 770772759874076672
index: 818, tweet_id: 770743923962707968
index: 819, tweet_id: 770655142660169732
index: 820, tweet_id: 770414278348247044
index: 821, tweet_id: 770293558247038976
index: 822, tweet_id: 770093767776997377
index: 823, tweet_id: 770069151037685760
index: 824, tweet_id: 769940425801170949
index: 825, tweet_id: 769695466921623552
index: 826, tweet_id: 769335591808995329
index: 827, tweet_id: 769212283578875904
index: 828, twee

...

index: 1592, tweet_id: 686394059078897668
index: 1593, tweet_id: 686386521809772549
index: 1594, tweet_id: 686377065986265092
index: 1595, tweet_id: 686358356425093120
index: 1596, tweet_id: 686286779679375361
index: 1597, tweet_id: 686050296934563840
index: 1598, tweet_id: 686035780142297088
index: 1599, tweet_id: 686034024800862208
index: 1600, tweet_id: 686007916130873345
index: 1601, tweet_id: 686003207160610816
index: 1602, tweet_id: 685973236358713344
index: 1603, tweet_id: 685943807276412928
index: 1604, tweet_id: 685906723014619143
index: 1605, tweet_id: 685681090388975616
index: 1606, tweet_id: 685667379192414208
index: 1607, tweet_id: 685663452032069632
index: 1608, tweet_id: 685641971164143616
index: 1609, tweet_id: 685547936038666240
index: 1610, tweet_id: 685532292383666176
index: 1611, tweet_id: 685325112850124800
index: 1612, tweet_id: 685321586178670592
index: 1613, tweet_id: 685315239903100929
index: 1614, tweet_id: 685307451701334016
index: 1615, tweet_id: 68526875363

index: 1788, tweet_id: 677557565589463040
index: 1789, tweet_id: 677547928504967168
index: 1790, tweet_id: 677530072887205888
index: 1791, tweet_id: 677335745548390400
index: 1792, tweet_id: 677334615166730240
index: 1793, tweet_id: 677331501395156992
index: 1794, tweet_id: 677328882937298944
index: 1795, tweet_id: 677314812125323265
index: 1796, tweet_id: 677301033169788928
index: 1797, tweet_id: 677269281705472000
index: 1798, tweet_id: 677228873407442944
index: 1799, tweet_id: 677187300187611136
index: 1800, tweet_id: 676975532580409345
index: 1801, tweet_id: 676957860086095872
index: 1802, tweet_id: 676949632774234114
index: 1803, tweet_id: 676948236477857792
index: 1804, tweet_id: 676946864479084545
index: 1805, tweet_id: 676942428000112642
index: 1806, tweet_id: 676936541936185344
index: 1807, tweet_id: 676916996760600576
index: 1808, tweet_id: 676897532954456065
index: 1809, tweet_id: 676864501615042560
index: 1810, tweet_id: 676821958043033607
index: 1811, tweet_id: 67681965106

index: 1984, tweet_id: 672902681409806336
index: 1985, tweet_id: 672898206762672129
index: 1986, tweet_id: 672884426393653248
index: 1987, tweet_id: 672877615439593473
index: 1988, tweet_id: 672834301050937345
index: 1989, tweet_id: 672828477930868736
index: 1990, tweet_id: 672640509974827008
index: 1991, tweet_id: 672622327801233409
index: 1992, tweet_id: 672614745925664768
index: 1993, tweet_id: 672609152938721280
index: 1994, tweet_id: 672604026190569472
index: 1995, tweet_id: 672594978741354496
index: 1996, tweet_id: 672591762242805761
index: 1997, tweet_id: 672591271085670400
index: 1998, tweet_id: 672538107540070400
index: 1999, tweet_id: 672523490734551040
index: 2000, tweet_id: 672488522314567680
index: 2001, tweet_id: 672482722825261057
index: 2002, tweet_id: 672481316919734272
index: 2003, tweet_id: 672475084225949696
index: 2004, tweet_id: 672466075045466113
index: 2005, tweet_id: 672272411274932228
index: 2006, tweet_id: 672267570918129665
index: 2007, tweet_id: 67226425178

index: 2181, tweet_id: 668994913074286592
index: 2182, tweet_id: 668992363537309700
index: 2183, tweet_id: 668989615043424256
index: 2184, tweet_id: 668988183816871936
index: 2185, tweet_id: 668986018524233728
index: 2186, tweet_id: 668981893510119424
index: 2187, tweet_id: 668979806671884288
index: 2188, tweet_id: 668975677807423489
index: 2189, tweet_id: 668967877119254528
index: 2190, tweet_id: 668960084974809088
index: 2191, tweet_id: 668955713004314625
index: 2192, tweet_id: 668932921458302977
index: 2193, tweet_id: 668902994700836864
index: 2194, tweet_id: 668892474547511297
index: 2195, tweet_id: 668872652652679168
index: 2196, tweet_id: 668852170888998912
index: 2197, tweet_id: 668826086256599040
index: 2198, tweet_id: 668815180734689280
index: 2199, tweet_id: 668779399630725120
index: 2200, tweet_id: 668655139528511488
index: 2201, tweet_id: 668645506898350081
index: 2202, tweet_id: 668643542311546881
index: 2203, tweet_id: 668641109086707712
index: 2204, tweet_id: 66863666581

NameError: name 'end' is not defined

In [17]:
errors

["TweepError: ([{'code': 144, 'message': 'No status found with that ID.'}],), tweeet_id: 888202515573088257",
 "TweepError: ([{'code': 144, 'message': 'No status found with that ID.'}],), tweeet_id: 873697596434513921",
 "TweepError: ([{'code': 144, 'message': 'No status found with that ID.'}],), tweeet_id: 872668790621863937",
 "TweepError: ([{'code': 144, 'message': 'No status found with that ID.'}],), tweeet_id: 872261713294495745",
 "TweepError: ([{'code': 144, 'message': 'No status found with that ID.'}],), tweeet_id: 869988702071779329",
 "TweepError: ([{'code': 144, 'message': 'No status found with that ID.'}],), tweeet_id: 866816280283807744",
 "TweepError: ([{'code': 144, 'message': 'No status found with that ID.'}],), tweeet_id: 861769973181624320",
 "TweepError: ([{'code': 144, 'message': 'No status found with that ID.'}],), tweeet_id: 856602993587888130",
 "TweepError: ([{'code': 34, 'message': 'Sorry, that page does not exist.'}],), tweeet_id: 856330835276025856",
 "TweepE

In [18]:
# Reading the 'tweet_json.txt' file into df_rt_fav
df_rt_fav = pd.DataFrame()

with open('tweet_json.txt', 'r') as f:
    for line in f:
        data = json.loads(line)
        tweet_json = {}
        tweet_json['tweet_id'] = data['id']
        tweet_json['retweet_count'] = data['retweet_count']
        tweet_json['favorite_count'] = data['favorite_count']
        
        df_rt_fav = df_rt_fav.append(tweet_json, ignore_index=True)

df_rt_fav.head(2)

,favorite_count,retweet_count,tweet_id
0,34463.0,7167.0,8.924206e+17
1,29874.0,5387.0,8.921774e+17


## Assessing Data
### `df_archive` accessing

In [19]:
# I have exported the `twitter_archive_enhanced.csv` to google sheet for better visualization
#
# Now trying to get more about the datatypes for each columns 
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [20]:
df_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [21]:
# I can see a couple of really high numerators and denominators, let's query this and export to google sheets for a better 
# visuals of this columns.
df_archive.query('rating_denominator > 10 or rating_numerator > 20').to_csv('data_issues.csv')

In [22]:
# Looking for duplicate tweets
df_archive.tweet_id.duplicated().value_counts()

False    2356
Name: tweet_id, dtype: int64

In [23]:
# View values in `source` column
df_archive.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

### `df_ipred` accessing

In [24]:
# Visualing a sample of df_ipred DataFrame
df_ipred.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
564,678255464182861824,https://pbs.twimg.com/media/CWmlvxJU4AEAqaN.jpg,1,Chihuahua,0.613819,True,Yorkshire_terrier,0.127931,True,Pomeranian,0.062124,True
441,674468880899788800,https://pbs.twimg.com/media/CVwx3dQXAAA0ksL.jpg,2,chow,0.526230,True,Pomeranian,0.283647,True,toy_poodle,0.067665,True
1318,755955933503782912,https://pbs.twimg.com/ext_tw_video_thumb/75595...,1,Pekinese,0.596882,True,Maltese_dog,0.176478,True,Great_Pyrenees,0.026775,True
265,670803562457407488,https://pbs.twimg.com/media/CU8sSAvXIAAB1Py.jpg,1,basenji,0.344101,True,Ibizan_hound,0.210282,True,toy_terrier,0.196279,True
79,667453023279554560,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,1,Labrador_retriever,0.825670,True,French_bulldog,0.056639,True,Staffordshire_bullterrier,0.054018,True
1657,810896069567610880,https://pbs.twimg.com/media/C0DhpcrUAAAnx88.jpg,1,flat-coated_retriever,0.820804,True,Labrador_retriever,0.082318,True,curly-coated_retriever,0.067461,True
577,678740035362037760,https://pbs.twimg.com/media/CWtede2WIAAF_AJ.jpg,1,seat_belt,0.787164,False,sunglasses,0.045739,False,beagle,0.022525,True
1568,794332329137291264,https://pbs.twimg.com/media/CwYJBiHXgAQlvrh.jpg,1,Samoyed,0.988307,True,malamute,0.004906,True,Great_Pyrenees,0.002901,True
194,669573570759163904,https://pbs.twimg.com/media/CUrNmtFWoAAnWCD.jpg,1,West_Highland_white_terrier,0.946828,True,miniature_schnauzer,0.022344,True,cairn,0.009462,True
87,667524857454854144,https://pbs.twimg.com/media/CUOGUfJW4AA_eni.jpg,1,hare,0.447893,False,dhole,0.092435,False,Chesapeake_Bay_retriever,0.088122,True


In [25]:
df_ipred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [26]:
df_ipred.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [30]:
# Looking for duplicate tweets
df_ipred.tweet_id.duplicated().value_counts()

False    2075
Name: tweet_id, dtype: int64

### `df_rt_fav` accessing

In [27]:
# Visualizing 10 sample from df_rt_fav DataFrame
df_rt_fav.sample(10)

,favorite_count,retweet_count,tweet_id
849,4949.0,1374.0,7.613340e+17
1431,2675.0,1023.0,6.950743e+17
1900,1295.0,579.0,6.740451e+17
101,30846.0,6990.0,8.721227e+17
1785,4639.0,1846.0,6.767764e+17
354,6134.0,1102.0,8.284097e+17
590,14152.0,4704.0,7.975452e+17
212,0.0,4559.0,8.496681e+17
1206,17972.0,6250.0,7.128090e+17
1594,3304.0,433.0,6.849270e+17


In [28]:
df_rt_fav.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2328 entries, 0 to 2327
Data columns (total 3 columns):
favorite_count    2328 non-null float64
retweet_count     2328 non-null float64
tweet_id          2328 non-null float64
dtypes: float64(3)
memory usage: 54.6 KB


In [29]:
df_rt_fav.describe()

,favorite_count,retweet_count,tweet_id
count,2328.000000,2328.000000,2.328000e+03
mean,7193.671821,2528.459192,7.418403e+17
std,11175.146664,4281.049883,6.823143e+16
min,0.000000,1.000000,6.660209e+17
25%,1247.500000,510.750000,6.781974e+17
50%,3114.500000,1177.500000,7.180382e+17
75%,8780.000000,2923.750000,7.986673e+17
max,147738.000000,72466.000000,8.924206e+17


### Quality issues
1. Drop retweets and replys

2. `NaN` values in columns `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` and `retweeted_status_timestamp` in `df_archive`.

3. Missing values in `expanded_urls` in `df_archive`.

4. `timestamp` is of dtype `string` instead of `date_time` in `df_archive`

5. `None` in columns `name`, `doggo`, `floofer`, `pupper` and `puppo` in `df_archive`.

6. `source` should be `categorical` instead of `string` dtype in `df_archive`.

6. Rate is 13/10 not 960/00 for tweet with id `835246439529840640` in `df_archive`.

7. Date for rating for tweet with id `832088576586297345` in `df_archive`.

8. 24/7 not a rate for tweet with id `810984652412424192` in `df_archive`.

9. Rate should be 9.75/10 not 75/10 for tweet with ids `[832215909146226688, 786709082849828864]` and 11.27/10 not 27/10 for tweet with id `778027034220126208`, 11.26/10 not 26/10 for tweet with id `680494726643068929` in `df_archive`.

10. Multiple rating for tweet with id `775096608509886464`, 9/11 and 14/10 in `df_archive`.

11. Tweet `740373189193256964` and `775096608509886464` are the same. `775096608509886464` is a retweet of `740373189193256964` in `df_archive`.

12. Multiple rating for tweet with id `722974582966214656` in `df_archive`.

13. 50/50 and 7/11 not a rate, rate should be 11/10 and 10/10 respectively for tweets with ids `716439118184652801` and `682962037429899265` in `df_archive`.

### Tidiness issues
1. `expanded_url` has multiple values in it.

2. `doggo`, `floofer`, `pupper` and `puppo` should be under one variable name `dog_stage`.

3. Merge `df_tweets_rt_fav` with `df_archive`.

4. `df_ipred` should be modified to `algorithm`, `prediction`, `confidence` and `is_dog`.

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [74]:
# Making copies of original dataframes
df_archive_copy = df_archive.copy()
df_ipred_copy = df_ipred.copy()
df_rt_fav_copy = df_rt_fav.copy()

### Issue #1:
#### Define: Drop retweets and reply
> Tweets with `in_reply_to_status_id` and `retweeted_status_id` are retweets and not original tweets so we can use this column to drop rows that are retweets.
>
> Afterwards we can drop the columns `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` and `retweeted_status_timestamp` since thay are all linked to retweets.

#### Code

In [75]:
# query for rows where in_reply_to_status_id and retweeted_status_id is not NaN
nan = np.nan

# NaN is not equal to NaN, then query for where in_reply_to_status_id == in_reply_to_status_id while omit the nan values
# This applies to retweeted_status_id
index_to_drop = df_archive_copy.query('in_reply_to_status_id == in_reply_to_status_id or retweeted_status_id == retweeted_status_id').index
df_archive_copy.drop(index=index_to_drop, inplace=True)

#### Test

In [76]:
df_archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2097 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2097 non-null object
source                        2097 non-null object
text                          2097 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2094 non-null object
rating_numerator              2097 non-null int64
rating_denominator            2097 non-null int64
name                          2097 non-null object
doggo                         2097 non-null object
floofer                       2097 non-null object
pupper                        2097 non-null object
puppo                         2097 non-null object
dtypes: float64(4), int64(3), object(10)

#### Code

In [77]:
# Awesom! Retweets dropped
#
# Now drop `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`
# and `retweeted_status_timestamp` columns since they are linked to retweets

columns_to_drop = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp']
df_archive_copy.drop(columns=columns_to_drop, inplace=True)

#### Test

In [78]:
df_archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
source                2097 non-null object
text                  2097 non-null object
expanded_urls         2094 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: int64(3), object(9)
memory usage: 213.0+ KB


### Issue #2:
#### Define: Missing values in `expanded_urls` in `df_archive`
> Drop rows with missing `expanded_urls` since these tweets have no image in them.

#### Code

In [79]:
# Get row index to drop
index_to_drop = df_archive_copy.query('expanded_urls != expanded_urls').index
df_archive_copy.drop(index=index_to_drop, inplace=True)

#### Test

In [80]:
df_archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2094 non-null int64
timestamp             2094 non-null object
source                2094 non-null object
text                  2094 non-null object
expanded_urls         2094 non-null object
rating_numerator      2094 non-null int64
rating_denominator    2094 non-null int64
name                  2094 non-null object
doggo                 2094 non-null object
floofer               2094 non-null object
pupper                2094 non-null object
puppo                 2094 non-null object
dtypes: int64(3), object(9)
memory usage: 212.7+ KB


### Issue #3:
#### Define: `timestamp` is of dtype `string` instead of `date_time` in `df_archive`
> Change `timestamp` datatype to `datetime`.

#### Code

In [81]:
df_archive_copy.timestamp = pd.to_datetime(df_archive_copy.timestamp)

#### Test

In [82]:
df_archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2094 non-null int64
timestamp             2094 non-null datetime64[ns]
source                2094 non-null object
text                  2094 non-null object
expanded_urls         2094 non-null object
rating_numerator      2094 non-null int64
rating_denominator    2094 non-null int64
name                  2094 non-null object
doggo                 2094 non-null object
floofer               2094 non-null object
pupper                2094 non-null object
puppo                 2094 non-null object
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 212.7+ KB


### Issue #4:
#### Define: `source` should be `categorical` instead of `string` dtype in `df_archive`
> Extract main string from `source` and the change datatype to `category`

#### Code

In [91]:
# Extracting main string from source

"""
ext_function: Extracts string from source.

Keyword arguments:
source_str -- The source sting from df_archive
"""
ext_function = lambda source_str: source_str[source_str.index('>') + 1:source_str.index('</')]

# Apply ext_function to df_archive.source
df_archive_copy.source = df_archive_copy.source.apply(ext_function)

In [92]:
df_archive_copy.source = df_archive_copy.source.astype('category')

#### Test

In [93]:
df_archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2094 non-null int64
timestamp             2094 non-null datetime64[ns]
source                2094 non-null category
text                  2094 non-null object
expanded_urls         2094 non-null object
rating_numerator      2094 non-null int64
rating_denominator    2094 non-null int64
name                  2094 non-null object
doggo                 2094 non-null object
floofer               2094 non-null object
pupper                2094 non-null object
puppo                 2094 non-null object
dtypes: category(1), datetime64[ns](1), int64(3), object(7)
memory usage: 198.5+ KB


In [94]:
df_archive_copy.source.value_counts()

Twitter for iPhone     1962
Vine - Make a Scene      91
Twitter Web Client       30
TweetDeck                11
Name: source, dtype: int64

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization